In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import sys
import torch

In [ ]:
sys.path.append("..")
plt.rcParams["figure.figsize"] = (3, 2)
mpl.rcParams['figure.dpi'] = 300

# Very simple 2-dimensional input example

In [ ]:
from easybo.gp import EasySingleTaskGPRegressor
from easybo.bo import ask
from easybo.logger import mode
from easybo.utils import get_dummy_2d_data, set_grids, grids_to_coordinates

**Step 1:** create some dummy training data.

In [ ]:
grid_x1, grid_x2, train_x, train_y, truth, truth_meshgrid = get_dummy_2d_data(seed=123)
grid = grids_to_coordinates([grid_x1, grid_x2])

**Step 2:** get the initial model conditioned on the training data, and run inference on the un-optimized GP, just to see what it looks like.

In [ ]:
model = EasySingleTaskGPRegressor(
    train_x=train_x,
    train_y=train_y,
    normalize_inputs_to_unity=True,
    standardize_outputs=True
)

**Step 3:** training: optimize the hyper-parameters (by default, this is just a kernel of the form `Const x RBF`. We can optionally use the `mode` context manager to indicate the logging level of the procedure. Note that this context manager can be used with any function, class, method, etc. in `easybo`.

In [ ]:
with mode(debug=True):
    model.train_()

**Step 4:** predict and plot!

In [ ]:
pred = model.predict(grid=grid)

In [ ]:
mu = pred["mean"].reshape(len(grid_x2), len(grid_x1))

In [ ]:
z = truth_meshgrid(grid_x1, grid_x2)
z_min = -np.abs(z).max()
z_max = np.abs(z).max()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(4, 2), sharey=True, sharex=True)

ax = axs[0]
c = ax.imshow(
    z.T, cmap='rainbow', vmin=z_min, vmax=z_max,
    extent=[grid_x1.min(), grid_x1.max(), grid_x2.min(), grid_x2.max()],
    interpolation ='nearest', origin ='lower'
)
set_grids(ax, grid=False)
ax.set_title("Function")

ax = axs[1]
c = ax.imshow(
    mu, cmap='rainbow', vmin=z_min, vmax=z_max,
    extent=[grid_x1.min(), grid_x1.max(), grid_x2.min(), grid_x2.max()],
    interpolation ='nearest', origin ='lower'
)
set_grids(ax, grid=False)
ax.scatter(train_x[:, 0], train_x[:, 1], s=0.3, color="black")
# ax.scatter(X_original[:, 0], X_original[:, 1], s=0.3, color="blue")
ax.set_title("GP", )

plt.show()

## Basic Bayesian optimization with constraints

We can do some simple Bayesian optimization under constraint. From the botorch docs:

> inequality_constraints (Optional[List[Tuple[Tensor, Tensor, float]]]) – A list of tuples (indices, coefficients, rhs), with each tuple encoding an inequality constraint of the form sum_i (X[indices[i]] * coefficients[i]) >= rhs

Let's do a simple constraint where we want $x_1 + x_2 <= -1$. Negating both sides to put it into the correct form, we have $-x_1 - x_2 >= 1,$ which would lead to a constraint `[(torch.tensor([0, 1]), torch.tensor([-1, -1]), 1)]`

In [ ]:
inequality_constraints = [(torch.tensor([0, 1]), torch.tensor([-1.0, -1.0]).float(), 1)]

In [ ]:
new_points = ask(
    model=model,
    bounds=[[-4, 5], [-5, 4]],
    acquisition_function="qMaxVar",
    acquisition_function_kwargs=dict(),
    optimize_acqf_kwargs={"q": 10, "num_restarts": 5, "raw_samples": 20, "inequality_constraints": inequality_constraints},
)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(4, 2), sharey=True, sharex=True)

ax = axs[0]
c = ax.imshow(
    z.T, cmap='rainbow', vmin=z_min, vmax=z_max,
    extent=[grid_x1.min(), grid_x1.max(), grid_x2.min(), grid_x2.max()],
    interpolation ='nearest', origin ='lower'
)
set_grids(ax, grid=False)
ax.set_title("Function")

ax = axs[1]
c = ax.imshow(
    mu, cmap='rainbow', vmin=z_min, vmax=z_max,
    extent=[grid_x1.min(), grid_x1.max(), grid_x2.min(), grid_x2.max()],
    interpolation ='nearest', origin ='lower'
)
set_grids(ax, grid=False)
ax.scatter(train_x[:, 0], train_x[:, 1], s=0.3, color="black")
ax.scatter(new_points[:, 0], new_points[:, 1], s=4, color="blue")
ax.set_title("GP")

plt.show()